In [1]:
"""
@author: Daehyeon

This file is made for calculating zscore. zscore means the average and the standard deviation.
So after calculate the zscore, it saves in pkl files.
But since there are too many input data, we calculate zscore using the samples of the whole data.
"""

import wave
import numpy as np
import python_speech_features as ps
import os
import glob
import cPickle
import random

import warnings
warnings.simplefilter("ignore", DeprecationWarning)

In [2]:
#wav files dir for *training*
wavFolder = "/mnt/home/20150132/Desktop/dataset/wav/train/"

wavfiles = [f for f in os.listdir(wavFolder) if os.path.isfile(os.path.join(wavFolder, f))]

In [3]:
len(wavfiles)

40132

In [4]:
def read_file(filename):
    file = wave.open(filename,'r')    
    params = file.getparams()
    nchannels, sampwidth, framerate, wav_length = params[:4]
    str_data = file.readframes(wav_length)
    wavedata = np.fromstring(str_data, dtype = np.short)
    time = np.arange(0,wav_length) * (1.0/framerate)
    file.close()
    
    return wavedata, time, framerate

In [5]:
def write_zscore(num):

    train_num = 0 # count the number of subsegments in total

    time_slice = 200
    #parameter
    filter_num = 40

    #save data in array
    melArr = np.array([])
    delta1Arr = np.array([])
    delta2Arr = np.array([])

    rdFiles = random.sample(range(len(wavfiles)), num) #randomly choose 'num' of files from wavfiles

    doneCount = 0
    for rdFile in rdFiles:
        if(doneCount % 100 == 0):
            print(doneCount)
        if((doneCount>= 1000) and (doneCount % 100 == 0)):
            melMean = np.mean(melArr,axis=0)
            melStd = np.std(melArr,axis=0)
            delta1Mean = np.mean(delta1Arr,axis=0)
            delta1Std = np.std(delta1Arr,axis=0)
            delta2Mean = np.mean(delta2Arr,axis=0)
            delta2Std = np.std(delta2Arr,axis=0)

            output = './zscore_2sec/zscore'+str(filter_num)+'_'+str(doneCount)+'.pkl'
            print(melMean, " " ,melStd," " ,delta1Mean," " ,delta1Std," " ,delta2Mean," " ,delta2Std)
            f=open(output,'wb') 
            cPickle.dump((melMean,melStd,delta1Mean,delta1Std,delta2Mean,delta2Std),f)
            f.close()  

        fileName = wavFolder+wavfiles[rdFile]

        data, _, rate = read_file(fileName)

        # get the log-mel value with delta and delta delta
        mel_spec = ps.logfbank(data,rate*2,nfilt = filter_num, nfft=1024)
        delta1 = ps.delta(mel_spec, 2)
        delta2 = ps.delta(delta1, 2)

        time = mel_spec.shape[0] # the time of audio

        # slice it to 3s subsegment
        frames = time/time_slice + 1
        for i in range(frames):
            begin = time_slice*i
            if(frames-1 == i): # for the last frame
                end =time
            else: 
                end= begin + time_slice

            # slice
            melt = mel_spec[begin:end,:]
            delta1t = delta1[begin:end,:]
            delta2t = delta2[begin:end,:]

            melArr = np.append(melArr, melt)
            delta1Arr = np.append(delta1Arr, delta1t)
            delta2Arr = np.append(delta2Arr, delta2t)

            train_num = train_num + 1

        doneCount+=1
        

    

In [6]:
write_zscore(40132)

0
100
200
300
400
500
600
700
800
900
1000
(4.855244083715198, ' ', 9.32036548370073, ' ', 0.05601946178979481, ' ', 0.7461951656637263, ' ', -0.0006736720953713724, ' ', 0.2695405201033613)
1100
(4.845599989595085, ' ', 9.355041851783794, ' ', 0.056448555871944604, ' ', 0.7483625959103023, ' ', -0.0006818057233501973, ' ', 0.2702344931982113)
1200
(4.859457567017858, ' ', 9.342678430941032, ' ', 0.056219862445524384, ' ', 0.7472089004022711, ' ', -0.0006673780361627742, ' ', 0.2699024056495453)
1300
(4.862491754015837, ' ', 9.347654555453412, ' ', 0.056364659644524424, ' ', 0.7473934846062964, ' ', -0.0006597848007323983, ' ', 0.26995054249648975)
1400
(4.870545523433876, ' ', 9.341825829436969, ' ', 0.05628429170103936, ' ', 0.7470002076058903, ' ', -0.0006536143858334772, ' ', 0.26980299221873616)
1500
(4.852328646275288, ' ', 9.349030296361013, ' ', 0.05650849075467932, ' ', 0.7474967856563075, ' ', -0.0006454384601201602, ' ', 0.27000826592173843)
1600
(4.850134384585123, ' ', 9.3

6300
(4.894284738165823, ' ', 9.311015131759664, ' ', 0.05571595844309911, ' ', 0.7459073338588869, ' ', -0.0006452444191305598, ' ', 0.26962970308604484)
6400
(4.894264268424001, ' ', 9.311745486314999, ' ', 0.055742067353192766, ' ', 0.7459267093601714, ' ', -0.0006439259053699416, ' ', 0.26963900205082736)
6500
(4.897606189713713, ' ', 9.313459037833747, ' ', 0.05576353146800008, ' ', 0.7459885003541719, ' ', -0.0006395393067754758, ' ', 0.2696671577227055)
6600
(4.895839959996782, ' ', 9.315671828953374, ' ', 0.0558256791773771, ' ', 0.7460442670332066, ' ', -0.0006360629193481406, ' ', 0.2697001574358706)
6700
(4.893340592903407, ' ', 9.31611835099353, ' ', 0.055837461212791116, ' ', 0.7461736722873262, ' ', -0.0006399235310382968, ' ', 0.2697277892480529)
6800
(4.890790704520752, ' ', 9.31958216681854, ' ', 0.05591469181198375, ' ', 0.7463696869943713, ' ', -0.0006361021479810211, ' ', 0.2697991196765337)
6900
(4.893327023797948, ' ', 9.320006430858358, ' ', 0.05590670791856259, 

11700
(4.895952444194345, ' ', 9.335107372270052, ' ', 0.056113657278225204, ' ', 0.7477283606507295, ' ', -0.0006354413941634327, ' ', 0.2703191456101891)
11800
(4.896441754319319, ' ', 9.33766314691374, ' ', 0.056152620350178636, ' ', 0.7478584073844822, ' ', -0.0006334531236783241, ' ', 0.2703684014972419)
11900
(4.896513489091107, ' ', 9.33836105340053, ' ', 0.05616868530954077, ' ', 0.7479526462728278, ' ', -0.0006324650118658122, ' ', 0.27039700853141774)
12000
(4.89592757222823, ' ', 9.33895419708135, ' ', 0.056188744559606575, ' ', 0.7479493288957197, ' ', -0.0006303442322586511, ' ', 0.27040381513146416)
12100
(4.897804654163452, ' ', 9.33729031217175, ' ', 0.056169272431721075, ' ', 0.7477972948356403, ' ', -0.0006289701692056814, ' ', 0.2703423091210309)
12200
(4.899986551485452, ' ', 9.334855585133933, ' ', 0.056100084335904944, ' ', 0.7476841282506294, ' ', -0.0006349429457248262, ' ', 0.2702912372787851)
12300
(4.901809609030524, ' ', 9.335366028165517, ' ', 0.05608049089

IOError: [Errno 122] Disk quota exceeded